In [1]:
import cv2, os, gc, csv, argparse
import tensorflow
import numpy as np
from os import path
from keras import Sequential
from keras.optimizers import SGD
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
import os
import glob
import numpy as np
from sklearn.utils import shuffle
from skimage.io import imread
from skimage.transform import resize
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, Dense, BatchNormalization
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from sklearn.preprocessing import StandardScaler

In [8]:
def create_conv(nb_cc_value):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(384, 683, 3)))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(MaxPool2D())
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPool2D())
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPool2D())
    model.add(BatchNormalization())
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(GlobalAveragePooling2D())
    model.add(Dense(nb_cc_value, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(9, activation='softmax'))  # 9 classes pour les labels de 0 à 8
    model.build((None, 384, 683, 3))

    lr_schedule = ExponentialDecay(initial_learning_rate=0.01, decay_steps=10000, decay_rate=0.7, staircase=False)
    optimizer = SGD(learning_rate=lr_schedule, momentum=0.9)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=[SparseCategoricalAccuracy()])
    return model


In [9]:
def load_data(data_dir, image_height, image_width):
    X = []
    y = []
    class_labels = os.listdir(data_dir)
    
    for label in class_labels:
        class_dir = os.path.join(data_dir, label)
        if os.path.isdir(class_dir):
            for file in glob.glob(os.path.join(class_dir, '*.png')):  # Changez l'extension si nécessaire
                image = imread(file)
                image_resized = resize(image, (image_height, image_width))
                X.append(image_resized)
                y.append(label)
    
    X = np.array(X)
    y = np.array(y)
    
    # Encoder les étiquettes en valeurs numériques si nécessaire
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    y = le.fit_transform(y)
    
    # Mélanger les données
    X, y = shuffle(X, y, random_state=42)
    
    return X, y
X_train, y_train = load_data('Dataset', 384, 683)

In [10]:
n_images = X_train.shape[0]
images_flattened = X_train.reshape(n_images, -1)
scaler = StandardScaler()
images_scaled = scaler.fit_transform(images_flattened)
X_train_scaled = images_scaled.reshape(X_train.shape)

In [11]:
model = create_conv(200)

In [17]:
model.fit(X_train_scaled, y_train, epochs = 400, batch_size = 8)

Epoch 1/400
8/8 [==============================] - 16s 1s/step - loss: 1.4892 - sparse_categorical_accuracy: 0.4762
Epoch 2/400
8/8 [==============================] - 3s 385ms/step - loss: 1.4695 - sparse_categorical_accuracy: 0.4127
Epoch 3/400
8/8 [==============================] - 3s 386ms/step - loss: 1.1297 - sparse_categorical_accuracy: 0.4921
Epoch 4/400
8/8 [==============================] - 3s 386ms/step - loss: 1.1638 - sparse_categorical_accuracy: 0.5079
Epoch 5/400
8/8 [==============================] - 3s 387ms/step - loss: 1.1262 - sparse_categorical_accuracy: 0.5556
Epoch 6/400
8/8 [==============================] - 3s 387ms/step - loss: 1.0672 - sparse_categorical_accuracy: 0.6032
Epoch 7/400
8/8 [==============================] - 3s 387ms/step - loss: 1.2412 - sparse_categorical_accuracy: 0.5397
Epoch 8/400
8/8 [==============================] - 3s 388ms/step - loss: 1.0528 - sparse_categorical_accuracy: 0.5397
Epoch 9/400
8/8 [==============================] - 3s 388m

In [23]:
prediction = model.predict(X_train_scaled, batch_size=1)

63/63 [==============================] - 2s 31ms/step


In [24]:
prediction = np.argmax(prediction, axis=1)

In [25]:
prediction

array([8, 8, 0, 6, 0, 5, 2, 1, 3, 8, 7, 1, 5, 1, 4, 7, 0, 2, 6, 1, 0, 4,
       0, 6, 2, 7, 5, 7, 4, 4, 8, 8, 2, 3, 3, 3, 6, 1, 4, 8, 6, 5, 4, 6,
       0, 3, 0, 7, 5, 5, 3, 7, 1, 3, 2, 8, 2, 1, 6, 2, 4, 7, 5],
      dtype=int64)

In [26]:
print(classification_report(y_train, prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00         7
           2       1.00      1.00      1.00         7
           3       1.00      1.00      1.00         7
           4       1.00      1.00      1.00         7
           5       1.00      1.00      1.00         7
           6       1.00      1.00      1.00         7
           7       1.00      1.00      1.00         7
           8       1.00      1.00      1.00         7

    accuracy                           1.00        63
   macro avg       1.00      1.00      1.00        63
weighted avg       1.00      1.00      1.00        63

